In [132]:
from uniswappy import *
import math

In [133]:
user_nm = 'user_intro'
eth_amount = 1000
dai_amount = 1000000

## (1) Derivation of Uni V2 swap 

In [134]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011")

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)
Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000.0, DAI = 1000000.0
Liquidity: 31622.776601683792 



Swap fee
>   $\gamma = 1 - 0.003 = \frac{997}{1000}$
>
Swap CPT formula with fee
> (Eq.1)  $ \quad (x - \Delta x)(y + \gamma \Delta y) = xy$
>
Isolate $\Delta x$
> (Eq.2)  $ \quad \Delta x = x - \frac{xy}{y + \gamma \Delta y}$
>
Some additional algebra
> (Eq.3) $ \quad \Delta x = \frac{\gamma x\Delta y}{y + \gamma \Delta y}$

### Perform swap using dervation

In [135]:
dy = 1000
gamma = 997/1000
x = lp.get_reserve(eth)
y = lp.get_reserve(dai)

dx = (gamma*x*dy)/(y + gamma*dy)

print(f'We receive {dx:.5f} ETH for {dy} DAI')

We receive 0.99601 ETH for 1000 DAI


### Perform swap using uniswappy

In [136]:
out = Swap().apply(lp, dai, user_nm, dy)
lp.summary()

print(f'We receive {out:.5f} ETH for {dy} DAI')

Exchange ETH-DAI (LP)
Reserves: ETH = 999.00399301896, DAI = 1001000.0
Liquidity: 31622.776601683792 

We receive 0.99601 ETH for 1000 DAI


## (2) Derivation of Uni V2 doubled-sided withdraw 

In [137]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011")

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)
Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000.0, DAI = 1000000.0
Liquidity: 31622.776601683792 



In [138]:
dx = 1
x = lp.get_reserve(eth)
y = lp.get_reserve(dai)
L = lp.get_liquidity()

Given CPT formula 
> $ xy = L^2 $
>

Price is determined as such
> $P = \frac{y}{x} = \frac{\Delta y}{\Delta x}$
>
> 
> $P = \frac{y}{x} = \frac{y^2}{xy} = \frac{y^2}{L^2}$, $\quad P = \frac{y}{x} = \frac{xy}{x^2} = \frac{L^2}{x^2}$
>  

Given P above, calculate $\Delta x$ and $\Delta y$ while maintaining price integrity

> $\sqrt{P} = \frac{y}{L}$, $\quad\quad\quad \sqrt{P} = \frac{L}{x}$
> 
> $\sqrt{\frac{\Delta y}{\Delta x}} = \frac{y}{L}$, $\quad\quad \sqrt{\frac{\Delta y}{\Delta x}} = \frac{L}{x}$
> 
> $\sqrt{\frac{\Delta y^2}{\Delta x \Delta y}} = \frac{y}{L}$, $\quad \sqrt{\frac{\Delta y \Delta x}{\Delta x^2}} = \frac{L}{x}$
>
> $\frac{\Delta y}{\Delta L} = \frac{y}{L}$, $\quad\quad\quad \frac{\Delta L}{\Delta x} = \frac{L}{x}$
>
> $\Delta y = \frac{y \Delta L}{L}$, $\quad\quad \Delta x = \frac{x \Delta L}{L}$
>

Take RHS of CPT formula and deduct $\Delta x$ and $\Delta y$ from both $x$ and $y$ to maintain integrity of the price 
> $ (x - \Delta x)(y - \Delta y) = (x - \frac{x \Delta L}{L})(y - \frac{y \Delta L}{L})$
>
> $\quad\quad\quad\quad\quad\quad\quad\quad = xy - 2xy\frac{\Delta L}{L} + xy\frac{\Delta L}{L} $
>  
> $\quad\quad\quad\quad\quad\quad\quad\quad = (L - \Delta L)^2$
>

CPT withdraw formula 
> $ (x - \Delta x)(y - \Delta y) = (L - \Delta L)^2 $

### Perform withdraw using dervation

In [139]:
dL = dx*L/x
dy = y*dL/L
new_x = (x-dx)
new_y = (y-dy) 
new_L = L-dL

print(f'The updated reserves are {new_x} ETH and {new_y} DAI, and the updated liquidity is {new_L:8f}')

The updated reserves are 999.0 ETH and 999000.0 DAI, and the updated liquidity is 31591.153825


### Perform withdraw using uniswappy

In [140]:
RemoveLiquidity().apply(lp, eth, user_nm, dx)
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 999.0, DAI = 999000.0
Liquidity: 31591.15382508211 



## (2) Derivation of Uni V2 doubled-sided deposit 

In [141]:
dai = ERC20("DAI", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", address="0x011")

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)
Join().apply(lp, user_nm, eth_amount, dai_amount)
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1000.0, DAI = 1000000.0
Liquidity: 31622.776601683792 



In [142]:
dx = 1
x = lp.get_reserve(eth)
y = lp.get_reserve(dai)
L = lp.get_liquidity()

We know from above to maintain price integrity, $\Delta x$ and $\Delta y$ are determined as such
> $\Delta y = \frac{y \Delta L}{L}$, $\quad\quad \Delta x = \frac{x \Delta L}{L}$
>
Take RHS of CPT formula and add $\Delta x$ and $\Delta y$ to both $x$ and $y$ to maintain integrity of the price 
> $ (x + \Delta x)(y + \Delta y) = (x + \frac{x \Delta L}{L})(y + \frac{y \Delta L}{L})$
>

Perform algebra simular to above and we get the CPT deposit formula 
> $ (x + \Delta x)(y + \Delta y) = (L + \Delta L)^2 $

### Perform deposit using dervation

In [143]:
dL = dx*L/x
dy = y*dL/L
new_x = (x+dx)
new_y = (y+dy) 
new_L = L+dL

print(f'The updated reserves are {new_x} ETH and {new_y} DAI, and the updated liquidity is {new_L:8f}')

The updated reserves are 1001.0 ETH and 1001000.0 DAI, and the updated liquidity is 31654.399378


### Perform deposit using uniswappy

In [144]:
AddLiquidity().apply(lp, eth, user_nm, dx)
lp.summary()

Exchange ETH-DAI (LP)
Reserves: ETH = 1001.0, DAI = 1001000.0
Liquidity: 31654.399378285478 

